In [ ]:
#| default_exp preprocesses

In [ ]:
#| hide
#| eval: false
# %%capture

!pip install git+https://github.com/huggingface/transformers.git



  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-xdhxuiuj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-xdhxuiuj
  Resolved https://github.com/huggingface/transformers.git to commit d90a36d192e2981a41122c30a765c63158dd0557
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 35.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.5/766.5 kB 53.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.22.0.dev0-py3-none-any.whl size=4730094 sha256=a6fdca27c5df1e5762ab68449646179d652168fc4bec7a4cc06babc5111efb5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-bs8sx32g/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers


In [ ]:
#| hide
#| export

from typing import List, Callable
from nbdev.showdoc import *
from IPython.display import display,SVG
from transformers import AutoFeatureExtractor
from datasets.dataset_dict import DatasetDict
from wav2keyword.datasets import dataloader_pipeline

# Code

In [ ]:
#| export

class Preprocessor(object):

    def __init__(self, model_checkpoint: str = "facebook/wav2vec2-base", max_duration: float = 1.0):
        self.model_checkpoint = model_checkpoint
        self.max_duration = max_duration
        self.FEATURE_EXTRACTOR = AutoFeatureExtractor.from_pretrained(model_checkpoint)

    def _preprocess_function(self, examples: List[dict]):
        """Runs the feature_extractor for the given checkpoint with max_duration.

        Args:
            examples (_type_): Audio example

        Returns:
            _type_: preprocessed example
        """
        audio_arrays = [x["array"] for x in examples["audio"]]
        inputs = self.FEATURE_EXTRACTOR(
            audio_arrays, 
            sampling_rate=self.FEATURE_EXTRACTOR.sampling_rate, 
            max_length=int(self.FEATURE_EXTRACTOR.sampling_rate * self.max_duration), 
            truncation=True,
        )
        return inputs

    def preprocess(self, dataset: DatasetDict):
        return dataset.map(self._preprocess_function, remove_columns=["audio", "file"], batched=True)


In [ ]:
show_doc(Preprocessor.preprocess)

# Examples

Before we can feed audio clips to our model, we need to preprocess them. This is done by 🤗 Transformers `FeatureExtractor` which will normalize the inputs and put them in a format the model expects, as well as generate the other inputs that the model requires.

We wrote a function that will preprocess our samples. The `_preprocess_function` is an internal function that will instantiate our feature extractor with the `AutoFeatureExtractor.from_pretrained` method, which will ensure that we get a preprocessor that corresponds to the model architecture we want to use.  
The argument `truncation=True` and the maximum sample length we will ensure that very long inputs like the ones in the `_silence_` class can be safely batched.

In [ ]:
#|filter_stream Reusing
#|filter_stream UserWarning
#| eval: false

data = dataloader_pipeline({'path': "superb", 'name': "ks"})
dataset = data['dataset']
sample = dataset['train'][:5]
preprocessor = Preprocessor()
preprocessed_sample = preprocessor._preprocess_function(sample)

Reusing dataset superb (/home/jovyan/.cache/huggingface/datasets/superb/ks/1.9.0/ce836692657f82230c16b3bbcb93eaacdbfd7de4def3be90016f112d68683481)


  0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:362: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
#| eval: false

sr = sample['audio'][0]['sampling_rate']
print(f"Raw sample durations in seconds: {[a['array'].shape[0]/sr for a in sample['audio']]}")
print(f"preprocessed sample durations in seconds: {[a.shape[0]/sr for a in preprocessed_sample['input_values']]}")


Raw sample durations in seconds: [95.183125, 61.8056875, 61.253875, 60.0, 61.1555]
preprocessed sample durations in seconds: [1.0, 1.0, 1.0, 1.0, 1.0]


To apply this function on all utterances in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

This whole process is mapped in the `preprocess` function. The results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

In [ ]:
#| filter_stream UserWarning|VisibleDeprecationWarning|Parameter|gradient_checkpointing|warnings.warn|tensor
#| eval: false

preprocess_dataset = preprocessor.preprocess(dataset)
preprocess_dataset

  0%|          | 0/52 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_values', 'label'],
        num_rows: 51094
    })
    validation: Dataset({
        features: ['input_values', 'label'],
        num_rows: 6798
    })
    test: Dataset({
        features: ['input_values', 'label'],
        num_rows: 3081
    })
})

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()